In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing

color = sns.color_palette()
%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

In [2]:
train = pd.read_csv("Desktop/Kaggle/train.csv",parse_dates=['timestamp'] )
test = pd.read_csv("Desktop/Kaggle/test.csv", parse_dates=['timestamp'])

In [3]:
first_feat = ["id","timestamp","price_doc", "full_sq", "life_sq",
"floor", "max_floor", "material", "build_year", "num_room",
"kitch_sq", "state", "product_type", "sub_area"]

In [4]:
bad_index = train[train.life_sq > train.full_sq].index
train.ix[bad_index, "life_sq"] = train.ix[bad_index,'full_sq']

In [5]:
bad_index = test[test.life_sq > test.full_sq].index
test.ix[bad_index, "life_sq"] = test.ix[bad_index,'full_sq']

In [6]:
bad_index = train[train.life_sq < 5].index
train.ix[bad_index, "life_sq"] = 5

In [7]:
bad_index = test[test.life_sq < 5].index
test.ix[bad_index, "life_sq"] = 5

In [8]:
bad_index = train[train.full_sq < 5].index
train.ix[bad_index, "full_sq"] = 5

In [9]:
bad_index = test[test.full_sq < 5].index
test.ix[bad_index, "full_sq"] = 5

In [10]:
kitch_is_build_year = [13117]
train.ix[kitch_is_build_year, "build_year"] = train.ix[kitch_is_build_year, "kitch_sq"]

In [11]:
bad_index = train[train.build_year < 1500].index
train.ix[bad_index, "build_year"] = np.NaN
bad_index = test[test.build_year < 1500].index
test.ix[bad_index, "build_year"] = np.NaN

In [12]:
bad_index = train[train.num_room == 0].index 
train.ix[bad_index, "num_room"] = np.NaN

In [13]:
bad_index = test[test.num_room == 0].index 
test.ix[bad_index, "num_room"] = np.NaN

In [14]:
bad_index = train[train.floor > train.max_floor].index
train.ix[bad_index, "floor"] = train.ix[bad_index, "max_floor"]

In [15]:
bad_index = test[test.floor > test.max_floor].index
test.ix[bad_index, "floor"] = test.ix[bad_index, "max_floor"]

In [16]:
bad_index = train[train.state == 33].index
train.ix[bad_index, "state"] = np.NaN

In [17]:
ulimit = np.percentile(train.price_doc.values, 99)
llimit = np.percentile(train.price_doc.values, 1)

In [18]:
train['price_doc'].ix[train['price_doc']>ulimit] = ulimit
train['price_doc'].ix[train['price_doc']<llimit] = llimit

In [19]:
train['life_sq'].isnull().sum()

6383

In [20]:
train['life_sq'].fillna(train['full_sq'],inplace = True)

In [21]:
for f in train.columns:
    if train[f].isnull().sum()>0:
        print(f)
        print(train[f].isnull().sum())

floor
167
max_floor
9572
material
9572
build_year
14507
num_room
9586
kitch_sq
9572
state
13560
preschool_quota
6688
school_quota
6685
hospital_beds_raion
14441
raion_build_count_with_material_info
4991
build_count_block
4991
build_count_wood
4991
build_count_frame
4991
build_count_brick
4991
build_count_monolith
4991
build_count_panel
4991
build_count_foam
4991
build_count_slag
4991
build_count_mix
4991
raion_build_count_with_builddate_info
4991
build_count_before_1920
4991
build_count_1921-1945
4991
build_count_1946-1970
4991
build_count_1971-1995
4991
build_count_after_1995
4991
metro_min_walk
25
metro_km_walk
25
railroad_station_walk_km
25
railroad_station_walk_min
25
ID_railroad_station_walk
25
cafe_sum_500_min_price_avg
13281
cafe_sum_500_max_price_avg
13281
cafe_avg_price_500
13281
cafe_sum_1000_min_price_avg
6524
cafe_sum_1000_max_price_avg
6524
cafe_avg_price_1000
6524
cafe_sum_1500_min_price_avg
4199
cafe_sum_1500_max_price_avg
4199
cafe_avg_price_1500
4199
cafe_sum_2000_min_

In [22]:
train['state'].fillna(train['state'].median(),inplace=True)

In [23]:
train['build_year'].fillna(train['build_year'].median(),inplace=True)

In [24]:
train['material'].unique()

array([ nan,   1.,   2.,   4.,   6.,   5.,   3.])

In [25]:
train['num_room'].unique()

array([ nan,   2.,   1.,   3.,   4.,   5.,   6.,  19.,  10.,   8.,   7.,
        17.,   9.])

In [26]:
bad_index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
train.ix[bad_index, "num_room"] = np.NaN

In [27]:
bad_index = [3174, 7313]
test.ix[bad_index, "num_room"] = np.NaN

In [28]:
train['num_room'].fillna(train['num_room'].median(),inplace=True)

In [29]:
test['state'].fillna(train['state'].median(),inplace=True)
test['build_year'].fillna(train['build_year'].median(),inplace=True)
test['num_room'].fillna(train['num_room'].median(),inplace=True)

In [30]:
for f in train.columns:
    if train[f].isnull().sum()>9000:
        print(f)
        print(train[f].isnull().sum())
        train.drop(f,axis=1,inplace=True)
        test.drop(f,axis=1,inplace=True)

max_floor
9572
material
9572
kitch_sq
9572
hospital_beds_raion
14441
cafe_sum_500_min_price_avg
13281
cafe_sum_500_max_price_avg
13281
cafe_avg_price_500
13281


In [31]:
for f in train.columns:
    if train[f].isnull().sum()>0:
        print(f)
        print(train[f].isnull().sum())
        train[f].fillna(train[f].median(),inplace=True)
        test[f].fillna(train[f].median(),inplace=True)

floor
167
preschool_quota
6688
school_quota
6685
raion_build_count_with_material_info
4991
build_count_block
4991
build_count_wood
4991
build_count_frame
4991
build_count_brick
4991
build_count_monolith
4991
build_count_panel
4991
build_count_foam
4991
build_count_slag
4991
build_count_mix
4991
raion_build_count_with_builddate_info
4991
build_count_before_1920
4991
build_count_1921-1945
4991
build_count_1946-1970
4991
build_count_1971-1995
4991
build_count_after_1995
4991
metro_min_walk
25
metro_km_walk
25
railroad_station_walk_km
25
railroad_station_walk_min
25
ID_railroad_station_walk
25
cafe_sum_1000_min_price_avg
6524
cafe_sum_1000_max_price_avg
6524
cafe_avg_price_1000
6524
cafe_sum_1500_min_price_avg
4199
cafe_sum_1500_max_price_avg
4199
cafe_avg_price_1500
4199
cafe_sum_2000_min_price_avg
1725
cafe_sum_2000_max_price_avg
1725
cafe_avg_price_2000
1725
cafe_sum_3000_min_price_avg
991
cafe_sum_3000_max_price_avg
991
cafe_avg_price_3000
991
prom_part_5000
178
cafe_sum_5000_min_price

In [32]:
train.shape

(30471, 285)

In [33]:
test.shape

(7662, 284)

In [34]:
train['ecology'].unique()

array(['good', 'excellent', 'poor', 'satisfactory', 'no data'], dtype=object)

In [35]:
train['ecology'][train['ecology']=='poor']=0

test['ecology'][test['ecology']=='poor']=0
train['ecology'][train['ecology']=='satisfactory']=1
test['ecology'][test['ecology']=='satisfactory']=1
train['ecology'][train['ecology']=='good']=3
test['ecology'][test['ecology']=='good']=3
train['ecology'][train['ecology']=='excellent']=4
test['ecology'][test['ecology']=='excellent']=4
train['ecology'][train['ecology']=='no data']=2
test['ecology'][test['ecology']=='no data']=2

In [36]:
train['ecology']=train['ecology'].astype('int64')
test['ecology']=test['ecology'].astype('int64')

In [37]:
for f in train.columns:
    if (train[f].dtype=='object') & ('ecology' not in f) & ('sub_area' not in f):
        print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values.astype('str')) + list(test[f].values.astype('str')))
        train[f] = lbl.transform(list(train[f].values.astype('str')))
        test[f] = lbl.transform(list(test[f].values.astype('str')))

product_type
culture_objects_top_25
thermal_power_plant_raion
incineration_raion
oil_chemistry_raion
radiation_raion
railroad_terminal_raion
big_market_raion
nuclear_reactor_raion
detention_facility_raion
water_1line
big_road1_1line
railroad_1line


In [38]:
train['sub_area']

0                         Bibirevo
1                Nagatinskij Zaton
2                   Tekstil'shhiki
3                           Mitino
4                        Basmannoe
5                   Nizhegorodskoe
6                       Sokol'niki
7                         Bibirevo
8                          Koptevo
9                          Kuncevo
10                Kosino-Uhtomskoe
11               Zapadnoe Degunino
12                         Kuncevo
13                     Presnenskoe
14                       Lefortovo
15                         Mar'ino
16                       Kuz'minki
17                        Nagornoe
18                Kosino-Uhtomskoe
19                      Gol'janovo
20                         Vnukovo
21                 Juzhnoe Tushino
22                Severnoe Tushino
23          Chertanovo Central'noe
24                 Juzhnoe Tushino
25                  Fili Davydkovo
26                        Otradnoe
27                Novo-Peredelkino
28                  

In [39]:
train['sub_area'].unique().shape

(146,)

In [40]:
train.shape

(30471, 285)

In [41]:
temp = pd.get_dummies(train)

In [45]:
test = pd.get_dummies(test)

In [46]:
test.shape

(7662, 428)

In [47]:
temp.shape

(30471, 430)

In [48]:
for f in temp.columns:
    if f not in test.columns:
        print(f)
    

price_doc
sub_area_Poselenie Klenovskoe


In [49]:
temp.drop('sub_area_Poselenie Klenovskoe',axis=1, inplace=True)

In [50]:
train = temp

In [51]:
train.shape

(30471, 429)

In [53]:
test.shape

(7662, 428)

In [55]:
for f in train.columns:
    if '_all' in f:
        train.drop(f,axis=1,inplace=True)
        test.drop(f,axis=1,inplace=True)

In [56]:
train.shape

(30471, 420)

In [57]:
test.shape

(7662, 419)

In [58]:
train["yearmonth"] = train["timestamp"].dt.year*100 + train["timestamp"].dt.month
test["yearmonth"] = test["timestamp"].dt.year*100 + test["timestamp"].dt.month

In [59]:
train["year"] = train["timestamp"].dt.year
test["year"] = test["timestamp"].dt.year


In [60]:
train["yearweek"] = train["timestamp"].dt.year*100 + train["timestamp"].dt.weekofyear
test["yearweek"] = test["timestamp"].dt.year*100 + test["timestamp"].dt.weekofyear

In [61]:
train["ratio_life_sq_full_sq"] = train["life_sq"] / train["full_sq"]
test["ratio_life_sq_full_sq"] = test["life_sq"] / train["full_sq"]

In [63]:
train["age_of_building"] = train["build_year"] - train["year"]
test["age_of_building"] = test["build_year"] - test["year"]

In [64]:
train.drop('year',axis=1,inplace = True)
test.drop('year',axis=1,inplace = True)

In [66]:
train["ratio_preschool"] = train["children_preschool"] / train["preschool_quota"]
test["ratio_preschool"] = test["children_preschool"] / test["preschool_quota"]


In [67]:
train['ratio_preschool']

0        1.914817
1        2.205835
2        4.018455
3        1.913584
4        1.761111
5        2.838028
6        2.635584
7        1.914817
8        1.988692
9        2.307901
10       1.966476
11       3.671989
12       2.307901
13       2.199074
14       2.824095
15       1.889238
16       1.705080
17       2.068012
18       1.966476
19       1.537592
20       1.349321
21       2.212412
22       2.475705
23       2.790259
24       2.212412
25       2.079486
26       1.916863
27       1.495411
28       2.286128
29       2.014459
           ...   
30441    2.260939
30442    0.171338
30443    2.109790
30444    1.913584
30445    2.564389
30446    0.229853
30447    2.109790
30448    0.524527
30449    2.564389
30450    1.916863
30451    0.136650
30452    4.073224
30453    1.180614
30454    2.286128
30455    0.096356
30456    1.913584
30457    2.014459
30458    0.524527
30459    2.109790
30460    2.567593
30461    1.776981
30462    2.564389
30463    2.475705
30464    1.795819
30465    2

In [68]:
train["ratio_school"] = train["children_school"] / train["school_quota"]
test["ratio_school"] = test["children_school"] / test["school_quota"]

In [69]:
train_X = train.drop(["id", "timestamp", "price_doc"], axis=1)
test_X = test.drop(["id", "timestamp"] , axis=1)

In [71]:
train_y = np.log1p(train['price_doc']+1)


In [73]:
train_X.to_csv("Desktop/Kaggle/train_clean_X.csv", index= False)
test_X.to_csv("Desktop/Kaggle/test_clean_X.csv", index = False)
train_y.to_csv("Desktop/Kaggle/train_clean_y.csv", index= False)
